## Every Dream trainer

You will need your data prepared first before starting!  Don't waste rental fees if you're not ready to upload your files.  Your files should be captioned before you start with either the caption as the filename or in text files for each image alongside the image files.  See main README.md for more details.

[Instructions](https://github.com/victorchall/EveryDream-trainer/blob/main/README.md)

If you can sign up for Runpod here: [Runpod](https://runpod.io?ref=oko38cd0)

If you are confused by the wall of text, join the discord here: [EveryDream Discord](https://discord.gg/uheqxU6sXN)

Make sure you have at least 40GB of Runpod **Volume** storage at a minimum so you don't waste training just 1 ckpt that is overtrained and have to start over.  Penny pinching on storage is ultimately a waste of your time and money!  The adage, "penny smart, pound foolish" applies here.  Get plenty of volume storage, train for a few epochs, and get a few ckpts you can try so you can pick the best one.  You can always delete the ckpts you don't want later.

Make sure you have your hugging face token ready. You can get one here: https://huggingface.co/settings/tokens
If you don't have a User Access Token, create one.

In [ ]:
!git clone https://github.com/victorchall/everydream-trainer

In [ ]:
%cd everydream-trainer
# should show /workspace/everydream-trainer below

## Install dependencies
You can ignore "warnings."

In [ ]:
!pip install albumentations==1.1.0
!pip install transformers==4.19.2
!pip install torchvision==0.13.1
!pip install pudb==2019.2
!pip install imageio==2.14.1
!pip install imageio-ffmpeg==0.4.7
!pip install test-tube>=0.7.5
!pip install einops==0.4.1
!pip install pillow==9.0.1
!pip install torch-fidelity==0.3.0
!pip install torchmetrics==0.6.0
!pip install kornia==0.6
!pip install huggingface_hub
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -e .

## Now that dependencies are installed, ready to move on!

## Log into huggingface
Run the cell below and paste your token into the prompt.  You can get your token from your huggingface account page.

The token will not show on the screen, just press enter after you paste it.

Then run the following cell to download the base checkpoint (may take a minute).

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="runwayml/stable-diffusion-v1-5",
 filename="v1-5-pruned.ckpt",
 use_auth_token=True
)

## Make an input folder for your training images

In [ ]:
!dir

In [ ]:
!mkdir input  #makes an input folder, UPLOAD YOUR TRAINING IMAGES THERE

# Upload training files

Ues the navigation on the left to upload your training files in the input folder.  Use the File menu to upload files.  You can upload multiple files at once.  You can also upload multiple folders under the input folder if you want.

You can check there are files in the folder by running the cell below (optional, just prints first 10).

In [10]:
!ls -U input | head -10

File Not Found
'head' is not recognized as an internal or external command,
operable program or batch file.


## Tweak your YAML
You can adjust the YAML file to change the training parameters.  

Instructions are here: https://github.com/victorchall/EveryDream-trainer/blob/main/README.md

[Runpod YAML](everydream-trainer/configs/stable-diffusion/v1-finetune_runpod.yaml) is a good starting point for small datasets (30-50 images) and is the default in the command below. It will only keep 2 checkpoints.

[EveryDream YAML](workspace/everydream-trainer/configs/stable-diffusion/v1-finetune_everydream.yaml) is a good starting point for large datasets. You will need to change the filename in the --config parameter below to use this.  This may create a LOT of large ckpt files while training, so make sure you have enough space in your runpod instance!  60GB+ is recommended.

# Run the trainer
This will take a while.  Make sure when it finishes you scroll down to run the last cell.

In [11]:
# run the trainer, wait until it finishes then SCROLL DOWN to the next cell
!python main.py --base configs/stable-diffusion/v1-finetune_micro.yaml -t  --actual_resume "v1-5-pruned.ckpt" -n test --data_root input

Running on GPUs 0,
Loading model from sd_v1-5_vae.ckpt
Instantiating config for: ldm.models.diffusion.ddpm.LatentDiffusion with config:
{'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'image', 'cond_stage_key': 'caption', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': True, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'unfreeze_model': True, 'model_lr': 1e-06, 'unet_config': {'target': 'ldm.modules.diffusionmodules.openaimodel.UNetModel', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'transformer_depth': 1, 'context_dim': 768, 'use_checkpoint': True, 'legacy': False}}, 'first_stage_config': {'target': 'ldm.models.autoencoder.AutoencoderKL', 'params': {'emb

Global seed set to 23
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.16.layer_norm1.bias', 'vision_model.encoder.layers.10.mlp.fc2.weight', 'vision_model.encoder.layers.7.self_attn.v_proj.weight', 'vision_model.encoder.layers.14.self_attn.k_proj.weight', 'vision_model.encoder.layers.6.self_attn.out_proj.weight', 'vision_model.encoder.layers.8.self_attn.q_proj.bias', 'vision_model.encoder.layers.2.mlp.fc1.weight', 'vision_model.encoder.layers.19.self_attn.q_proj.weight', 'vision_model.encoder.layers.18.layer_norm2.weight', 'vision_model.encoder.layers.15.layer_norm1.bias', 'vision_model.encoder.layers.15.layer_norm2.weight', 'vision_model.encoder.layers.22.layer_norm2.bias', 'vision_model.encoder.layers.19.self_attn.v_proj.weight', 'vision_model.encoder.layers.8.self_attn.k_proj.weight', 'vision_model.encoder.layers.12.self_attn.q_proj.weight', 'vision_model.encoder.layers.12.mlp.fc1.bias

## prune checkpoints
This will create 2GB pruned files for all your 

## Prune your checkpoints
This will create 2GB pruned files for all your checkpoints. 

In [ ]:
# prune the ckpts
!python auto_prune_all.py --delete

## Download your checkpoints

Use the file explorer on the left and look for all the ckpt files that say "-pruned" on the end.  Download them and you're done! 

[EveryDream Discord](https://discord.gg/uheqxU6sXN)